# ADK 101 in 30min


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/QuickStart.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FQuickStart.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/QuickStart.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/blob/main/QuickStart.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Pili Hu](https://github.com/hupili)|
| Reviewers(s) | Nil |
| Last updated | 2025-06-08 |
| google-adk | 1.1.1 |

# Environment


In [13]:
!pip3 install deprecated
!pip3 install google-adk==1.1.1

In [64]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "global" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API
# [your-project-id]

In [65]:
from google.colab import auth
auth.authenticate_user()

# Utils

In [26]:
import json
import time

def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()
    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()
        agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))

        if is_final_response:
            print('>>> inside final response...')
            final_response = event.content.parts[0].text
            end_time = time.time()
            elapsed_time_ms = round((end_time - start_time) * 1000, 3)
            print(f'Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-----------------------------")
        elif function_calls:
            print('+++ inside the function call...')
            for function_call in function_calls:
                print(f"function, [args]:  {function_call.name}, {function_call.args}")
        elif function_responses:
            print('--- inside the function call response...')
            # TODO(Pili): copied from walkthrough codes. Find root cause of 'pending' not found.
            # if not event.actions.pending:
            if True:
                for function_response in function_responses:
                    details = function_response.response
                    recommended_list = list(details.values())
                    print(f"Function Name: {function_response.name}")
                    result=json.dumps(recommended_list)
                    print(f"Function Results {result}")
            else:
                print(agent_res)
    print(f"Total elapsed time: {elapsed_time_ms}")


In [27]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='google.generativeai.types.content_types') # Suppress harmless warning

In [28]:
from google import adk

adk.__version__

'1.1.1'

In [29]:
from google.adk.agents import Agent
from google.genai import types
from google.adk.tools.agent_tool import AgentTool

In [30]:
MODEL = "gemini-2.0-flash"

# Hello world!

Concepts:

- Agent - Starts as a wrapper of LLM that can decide actions and interact with external world (other agents, tools)
- Session - Maintains the state of one invocation. (chat log / events/ etc)
- Runner - The orchestration layer of agents. The "state machine". Runner drives agents to move the states of a session forward.

Use case:
- An agent that says hello word in a random language

In [31]:
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

In [32]:
# Constant
APP_NAME = "hello_world_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_word_agent"

# Agent
hello_world_agent = Agent(
    model=MODEL,
    name="hello_world_agent",
    description="An agent that says 'hello world'",
    instruction="""You always say 'hello world' to the user, and nothing else.
    Output 'hello world' in a random language.
    Put the language in brackets.

    Example Output 1:
    hello world (English)

    Example Output 2:
    你好，世界 (Chinese)
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_world_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
  return events

In [33]:
events = call_agent(runner, "hello")

In [34]:
list(events)

[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='Hallo Welt (German)\n')], role='model'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=6, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=6)], prompt_token_count=99, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=99)], thoughts_token_count=None, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=None, total_token_count=105, traffic_type=<TrafficType.ON_DEMAND: 'ON_DEMAND'>), invocation_id='e-56382b2e-b0fe-4ae7-b8d3-1443ca41fff1', auth

In [35]:
events = call_agent(runner, "hello")
pprint_events(events)

>>> inside final response...
Final Response (761.95 ms):
Ciao mondo (Italian)

-----------------------------
Total elapsed time: 761.95


In [36]:
events = call_agent(runner, "hello")
pprint_events(events)

>>> inside final response...
Final Response (896.333 ms):
Bonjour le monde (French)

-----------------------------
Total elapsed time: 896.333


In [38]:
events = call_agent(runner, "say something other then hello world.")
pprint_events(events)

>>> inside final response...
Final Response (897.47 ms):
hello world (English)

-----------------------------
Total elapsed time: 897.47


In [39]:
events = call_agent(runner, "1")
pprint_events(events)

>>> inside final response...
Final Response (983.397 ms):
hello world (English)

-----------------------------
Total elapsed time: 983.397


In [40]:
events = call_agent(runner, "say 'goodbye world'")
pprint_events(events)

>>> inside final response...
Final Response (866.549 ms):
hello world (English)

-----------------------------
Total elapsed time: 866.549


In [41]:
events = call_agent(runner, "")
pprint_events(events)

>>> inside final response...
Final Response (950.946 ms):
hello world (English)

-----------------------------
Total elapsed time: 950.946


In [42]:
session_list = await session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)
session_list

ListSessionsResponse(sessions=[Session(id='session12345', app_name='hello_world_example', user_id='user12345', state={}, events=[], last_update_time=1749350203.736096)])

In [43]:
from pprint import pprint

pprint(session_service.sessions[APP_NAME][USER_ID][SESSION_ID].events)

[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-56382b2e-b0fe-4ae7-b8d3-1443ca41fff1', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='RfU5SdnT', timestamp=1749349875.359099),
 Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='Hallo Welt (German)\n')], role='model'), grounding_metadat

In [44]:
events = session_service.sessions[APP_NAME][USER_ID][SESSION_ID].events

In [45]:
for event in events:
  print(f"Role: {event.content.role}, Text: {event.content.parts[0].text if event.content and event.content.parts else 'N/A'}")

Role: user, Text: hello
Role: model, Text: Hallo Welt (German)

Role: user, Text: hello
Role: model, Text: Ciao mondo (Italian)

Role: user, Text: hello
Role: model, Text: Bonjour le monde (French)

Role: user, Text: hello
Role: model, Text: Hola mundo (Spanish)

Role: user, Text: say something other then hello world.
Role: model, Text: hello world (English)

Role: user, Text: 1
Role: model, Text: hello world (English)

Role: user, Text: say 'goodbye world'
Role: model, Text: hello world (English)

Role: user, Text: 
Role: model, Text: hello world (English)



# Multi turn conversation

Concepts:

- Event(s)
- Session
- SessionService: sessions[app][user][session]

Use case:

- An agent that tries to know your name and send hello USERNAME

In [46]:
# Constant
APP_NAME = "hello_name_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_name_agent"

# Agent
hello_name_agent = Agent(
    model=MODEL,
    name="hello_name_agent",
    description="An agent that says 'hello USERNAME'",
    instruction="""
    You need to first ask the user's name.
    Try best to convince the user to give you a name, let it be first name, last name, or nick name.

    Once you get the user's name, say 'hello USERNAME'.
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_name_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, session, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
  return events

events = call_agent(runner, session, "hello")
pprint_events(events)

>>> inside final response...
Final Response (1073.74 ms):
Hello there! Before I say hello back properly, could I get your name? It can be your first name, last name, or even a nickname! I'd love to know what to call you.

-----------------------------
Total elapsed time: 1073.74


In [47]:
events = call_agent(runner, session, "I don't tell you my name")
pprint_events(events)

>>> inside final response...
Final Response (1057.184 ms):
Okay, I understand you might be hesitant. But knowing your name would help me personalize our interaction. How about just a first name or a nickname? It would make saying hello much more friendly. Pretty please? 😊

-----------------------------
Total elapsed time: 1057.184


In [48]:
events = call_agent(runner, session, "what is your name?")
pprint_events(events)

>>> inside final response...
Final Response (1005.11 ms):
You can call me hello_name_agent. But, I'm still curious, what should I call you? Just a little something?

-----------------------------
Total elapsed time: 1005.11


In [49]:
events = call_agent(runner, session, "my name is Pili")
pprint_events(events)

>>> inside final response...
Final Response (835.701 ms):
Hello Pili! It's nice to meet you.

-----------------------------
Total elapsed time: 835.701


In [50]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> inside final response...
Final Response (1050.806 ms):
Your name is Pili! I remembered! 😊

-----------------------------
Total elapsed time: 1050.806


## Switch session

In [51]:
new_session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID + '-new')

In [52]:
events = call_agent(runner, new_session, "what is my name?")
pprint_events(events)

>>> inside final response...
Final Response (1170.207 ms):
I don't know your name yet! Could you please tell me what I should call you? It can be your first name, last name, or even a nickname! 

-----------------------------
Total elapsed time: 1170.207


In [53]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> inside final response...
Final Response (931.079 ms):
Pili! You told me your name is Pili. Is there a different name you'd like me to use?

-----------------------------
Total elapsed time: 931.079


# Use Built-in Tool

In [66]:
from google.adk.tools import google_search

# Constant
APP_NAME = "hello_search_example"
USER_ID = "user123456"
SESSION_ID = "session123456"
AGENT_NAME = "hello_search_agent"

# Agent
hello_search_agent = Agent(
    # model=MODEL,
    model='gemini-2.5-pro-preview-06-05',
    name="hello_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

# Session and Runner
session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_search_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, session, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
  return events

events = call_agent(runner, session, "what is last Google IO date?")
pprint_events(events)

>>> inside final response...
Final Response (8823.17 ms):
The last Google I/O was held on **May 20 and 21, 2025**.

The annual developer conference took place at the Shoreline Amphitheatre in Mountain View, California, and was also available to stream online. Key announcements from the event included updates on Android, artificial intelligence (AI), web technologies, and cloud services. A significant focus was on Google's Gemini AI models and the introduction of Android 16.
-----------------------------
Total elapsed time: 8823.17


In [67]:
events = call_agent(runner, session, "what is last Google IO date?")


In [68]:
list(events)

[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="The most recent Google I/O took place on May 20 and 21, 2025. The annual developer conference was held at the Shoreline Amphitheatre in Mountain View, California, and was also streamed online for virtual attendees.\n\nKey topics and announcements from the event included:\n*   **Artificial Intelligence**: A major focus was on AI, with updates to Google's Gemini models. The company also introduced an AI coding agent named Jules and Veo 3, a video generation model that can create corresponding audio.\n*   **Android 16**: Details about the next version of Android were unveiled, including potential changes to the user interface, volume controls, and enhanced privacy and security settings.\n*   **Android XR**: Google and Samsung are expected to introduce a new extende

In [ ]:
events = call_agent(runner, session, "when is cloud summit Hong Kong?")
pprint_events(events)

In [57]:
events = call_agent(runner, session, "what is the weather now in Hong Kong?")
pprint_events(events)

>>> inside final response...
Final Response (2153.882 ms):
The weather in Hong Kong is currently around 29°C. It is sunny with a gentle breeze. The humidity is at 81% and the pressure is 1007mb, rising.

-----------------------------
Total elapsed time: 2153.882


In [59]:
events = call_agent(runner, session, "write a short CV of Eistein")
pprint_events(events)

>>> inside final response...
Final Response (2558.097 ms):
**Albert Einstein**

**Born:** March 14, 1879, Ulm, Germany
**Died:** April 18, 1955, Princeton, New Jersey, USA
**Nationality:** German, Swiss, American

**Education:**

*   Swiss Federal Polytechnic, Zurich (Diploma in Physics, 1900)
*   University of Zurich (PhD, 1905)

**Career:**

*   Swiss Patent Office (1902-1909)
*   University of Zurich (Professor, 1909-1911)
*   Charles University in Prague (Professor, 1911-1912)
*   ETH Zurich (Professor, 1912-1914)
*   Kaiser Wilhelm Institute for Physics, Berlin (Director, 1914-1933)
*   University of Berlin (Professor, 1914-1933)
*   Institute for Advanced Study, Princeton (Professor, 1933-1955)

**Key Achievements:**

*   Special Relativity (1905)
*   General Relativity (1915)
*   Explanation of the Photoelectric Effect
*   Brownian Motion
*   E=mc²
*   Nobel Prize in Physics (1921)

**Skills:** Theoretical Physics, Mathematical Physics.

-----------------------------
Total elaps

In [69]:
events = call_agent(runner, session, "write a short CV of Eistein in markdown bullet point list")
pprint_events(events)

>>> inside final response...
Final Response (14637.052 ms):
Of course, here is a short CV of Albert Einstein in markdown bullet points: 

 ### **Albert Einstein** 

 (1879-1955) 

 #### **Profile** 

 A German-born theoretical physicist, widely acknowledged as one of the most influential scientists of all time. Best known for developing the theory of relativity, Einstein's work also made significant contributions to the development of the theory of quantum mechanics. 

 #### **Education** 

 *   **1905:** Doctorate from the University of Zurich. 
 *   **1900:** Diploma in mathematics and physics from the Swiss Federal Polytechnic School (now ETH Zurich). 

 #### **Professional Experience** 

 *   **1933-1955:** Professor at the Institute for Advanced Study in Princeton, New Jersey. 
 *   **1914-1933:** Professor at the University of Berlin and member of the Prussian Academy of Sciences. He was also the Director of the Kaiser Wilhelm Institute for Physics. 
 *   **1911-1912:** Professor

In [70]:
events = call_agent(runner, session, "write a short CV of Pili Hu in markdown bullet point list")
pprint_events(events)

>>> inside final response...
Final Response (19739.768 ms):
Here is a short CV of Pili Hu in a markdown bullet point list:

### **Pili Hu**

#### **Profile**

A technology professional with a background in data science, artificial intelligence, and software engineering. He has experience across academia and industry, with a recent focus on Generative AI. He is also an accomplished ultra-runner and founder of a running community.

#### **Professional Experience**

*   **Google:** Currently a Generative AI Global Blackbelt (also referred to as a GenAI Specialist). He has also been involved in delivering workshops on Google Cloud Platform components.
*   **Initium Media:** Formerly the Chief Technology Officer.
*   **Hong Kong Baptist University:** Previously a lecturer in the Department of Journalism.
*   **Baidu:** Worked as a search engine algorithm research and development engineer.

#### **Education**

*   **The Chinese University of Hong Kong (CUHK):** PhD candidate in the Departmen

# Use Function Tool

Concept:
- Tool (Python function) -- the experience of tool calling by agent (by LLM) or by human (from Python) are the same. Easy to test.

Use case:
- Simple math

In [71]:
def add(numbers: list[int]) -> int:
  """Calculates the sum of a list of integers.

    This function takes a list of integers as input and returns the sum of all
    the elements in the list.  It uses the built-in `sum()` function for
    efficiency.

    Args:
        numbers: A list of integers to be added.

    Returns:
        The sum of the integers in the input list.  Returns 0 if the input
        list is empty.

    Examples:
        add([1, 2, 3]) == 6
        add([-1, 0, 1]) == 0
        add([]) == 0
    """
  return sum(numbers)

def subtract(numbers: list[int]) -> int:
    """Subtracts numbers in a list sequentially from left to right.

    This function performs subtraction on a list of integers, applying the
    subtraction operation from left to right.  For example, given the list
    [10, 2, 5], the function will calculate 10 - 2 - 5.

    Args:
        numbers: A list of integers to be subtracted.

    Returns:
        The result of the sequential subtraction as an integer. Returns 0 if the input list is empty.

    Examples:
        subtract([10, 2, 5]) == 3  # (10 - 2) - 5 = 8 - 5 = 3
        subtract([10, 2]) == 8      # 10 - 2 = 8
        subtract([]) == 0
    """
    if not numbers:
        return 0  # Handle empty list
    result = numbers[0]
    for num in numbers[1:]:
        result -= num
    return result

def multiply(numbers: list[int]) -> int:
  """Calculates the product of a list of integers.

    This function takes a list of integers as input and returns the product of all
    the elements in the list. It iterates through the list, multiplying each
    number with the accumulated product.

    Args:
        numbers: A list of integers to be multiplied.

    Returns:
        The product of the integers in the input list. Returns 1 if the input
        list is empty.

    Examples:
        multiply([2, 3, 4]) == 24  # 2 * 3 * 4 = 24
        multiply([1, -2, 5]) == -10 # 1 * -2 * 5 = -10
        multiply([]) == 1
    """
  product = 1
  for num in numbers:
    product *= num
  return product

def divide(numbers: list[int]) -> float:  # Use float for division
    """Divides numbers in a list sequentially from left to right.

    This function performs division on a list of integers, applying the division
    operation from left to right.  For example, given the list [10, 2, 5], the
    function will calculate 10 / 2 / 5.

    Args:
        numbers: A list of integers to be divided.

    Returns:
        The result of the sequential division as a float.

    Raises:
        ZeroDivisionError: If any number in the list *after* the first element
                           is zero, a ZeroDivisionError is raised.  Division by
                           zero is not permitted.

    Returns:
        float: The result of the division. Returns 0.0 if the input list is empty.

    Examples:
        divide([10, 2, 5]) == 1.0  # (10 / 2) / 5 = 5 / 5 = 1.0
        divide([10, 2]) == 5.0      # 10 / 2 = 5.0
        divide([10, 0, 5])  # Raises ZeroDivisionError
        divide([]) == 0.0
    """
    if not numbers:
        return 0.0 # Handle empty list
    if 0 in numbers[1:]: # Check for division by zero
        raise ZeroDivisionError("Cannot divide by zero.")
    result = numbers[0]
    for num in numbers[1:]:
        result /= num
    return result


In [73]:
add([1, 2, 3, 4, 5])

15

In [74]:
multiply([2, 2, 3])

12

In [78]:
simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

In [79]:
import random

async def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
  session_service = InMemorySessionService()
  if session_id is None:
    suffix = random.randint(100000, 999999)
    session_id = f'{app_name}-{user_id}-{suffix}'
  session = await session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id)
  runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service)
  def _call(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
    return events
  return _call

In [80]:
call = await caller_factory(root_agent=simple_math_agent)

In [81]:
pprint_events(call('hello'))

>>> inside final response...
Final Response (1184.858 ms):
Hello! How can I help you with your math today?

-----------------------------
Total elapsed time: 1184.858


In [82]:
pprint_events(call('what is three plus 9?'))

+++ inside the function call...
function, [args]:  add, {'numbers': [3, 9]}
--- inside the function call response...
Function Name: add
Function Results [12]
>>> inside final response...
Final Response (1762.321 ms):
The answer is 12.

-----------------------------
Total elapsed time: 1762.321


In [83]:
pprint_events(call('multiply that by 2'))

+++ inside the function call...
function, [args]:  multiply, {'numbers': [12, 2]}
--- inside the function call response...
Function Name: multiply
Function Results [24]
>>> inside final response...
Final Response (2110.523 ms):
The answer is 24.

-----------------------------
Total elapsed time: 2110.523


In [84]:
call = await caller_factory(root_agent=simple_math_agent)
pprint_events(call('Here is my math problem that is about apple counting. Let us start saying that I have three apple'))

>>> inside final response...
Final Response (1224.942 ms):
Okay. I understand you have three apples. What would you like to do with that number? Do you want to add more apples, subtract some, multiply, or divide? For example, you could say "add 5 apples" or "multiply by 2".

-----------------------------
Total elapsed time: 1224.942


In [85]:
pprint_events(call('Alice gave anoter 2 apples'))

+++ inside the function call...
function, [args]:  add, {'numbers': [3, 2]}
--- inside the function call response...
Function Name: add
Function Results [5]
>>> inside final response...
Final Response (1912.267 ms):
Okay, now you have 5 apples.

-----------------------------
Total elapsed time: 1912.267


In [86]:
pprint_events(call('how many apples do I have?'))

>>> inside final response...
Final Response (754.644 ms):
You have 5 apples.

-----------------------------
Total elapsed time: 754.644


In [87]:
pprint_events(call('Bob gave me 3 apples. how many apples do I have?'))

+++ inside the function call...
function, [args]:  add, {'numbers': [5, 3]}
--- inside the function call response...
Function Name: add
Function Results [8]
>>> inside final response...
Final Response (1732.069 ms):
Now you have 8 apples.

-----------------------------
Total elapsed time: 1732.069


In [88]:
pprint_events(call('Alice and Bob do this to me for three more times. how many apples do I have?'))

+++ inside the function call...
function, [args]:  add, {'numbers': [8, 5, 5, 5]}
--- inside the function call response...
Function Name: add
Function Results [23]
>>> inside final response...
Final Response (2106.363 ms):
After Alice and Bob give you more apples three times, you will have 23 apples.

-----------------------------
Total elapsed time: 2106.363


# Agent As Tool

Tool: If the execution flow goes from Agent to Tool and it always come back to the initiation agent.

Agent: If the execution flow goes from Agent to **another** agent, the latter agent can decide where the flow goes next.

Concepts:
- AgentTool -- If you want another agent to handle a task and always come back to the caller, make this `Agent` a `Tool` by `AgentTool`

Use case:
- Advanced math agent

In [89]:
agent_math_advanced_instruction = '''
I am an advanced math agent. I handle user query in the below steps:

1. I shall analyse the chat log to understand current question and make a math formula for it.
2. Break down a complex compuation based on arithmetic priority and hand over to simple_math_agent for the calculation.
3. Note that simple_math_agent can only understand numbers, so I need to convert natural language expression of numbers into digits.

<example>
<input> alice gives us 3 apples, bob gives us 5 apples. They do this seven times. Then we eat four apples. How many apples do we have now? </input>
<think> what is (3+5) * 7 -4 </think>
<think>I need to first calculate (3+5) as the highest priority operation.</think>
<call_tool> pass (3+5) to simple_math_agent </call_tool>
<tool_response>8</tool_response>
<think> The question now becomes 8 * 7 - 4, and next highest operation is 8 * 7</think>
<call_tool> pass 8 * 7 to simple_math_agent </call_tool>
<tool_response>56</tool_response>
<think> The question now becomes 56 - 4, and next highest operation is 56 - 4</think>
<call_tool> pass 56 - 4 to simple_math_agent </call_tool>
<tool_response>52</tool_response>
<think>There is a single number, so it is the final answer.</think>
<output>The result of "(3+5) * 7 - 4" is 52</output>
</example>
'''

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [90]:
call = await caller_factory(root_agent=agent_math_advanced)

In [91]:
pprint_events(call('who are you?'))

>>> inside final response...
Final Response (1208.616 ms):
I am an advanced math agent. I handle user query in the below steps:

1. I shall analyse the chat log to understand current question and make a math formula for it.
2. Break down a complex compuation based on arithmetic priority and hand over to simple_math_agent for the calculation.
3. Note that simple_math_agent can only understand numbers, so I need to convert natural language expression of numbers into digits.

-----------------------------
Total elapsed time: 1208.616


In [92]:
pprint_events(call('what is 1 + (three+2) times 7 '))

+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '3+2'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The answer is 5.\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '5*7'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["35\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '1+35'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["36\n"]
>>> inside final response...
Final Response (8640.365 ms):
The result of "1 + (3+2) times 7" is 36

-----------------------------
Total elapsed time: 8640.365


In [93]:
call = await caller_factory(root_agent=agent_math_advanced)

In [94]:
pprint_events(call('add from 1 to 2. then multiply that by 2.'))

+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '1+2'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The answer is 3.\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '3*2'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The answer is 6.\n"]
>>> inside final response...
Final Response (6233.233 ms):
The result of "add from 1 to 2. then multiply that by 2" is 6

-----------------------------
Total elapsed time: 6233.233


In [95]:
call = await caller_factory(root_agent=agent_math_advanced)
pprint_events(call('how much is 10 / 5 + three times 5?'))

+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '10 / 5'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The result is 2.0\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '3 * 5'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["15\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2 + 15'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The answer is 17.\n"]
>>> inside final response...
Final Response (8539.179 ms):
The result of "10 / 5 + three times 5" is 17

-----------------------------
Total elapsed time: 8539.179


**NOTE**: This part is very nice, it shows how two agents interact to clarify the input type should be integers and input again.

```
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2.0 + 15'}
--- inside the function call response...
Function Name: simple_math_agent
Function Results ["I can only work with integers. Could you provide the numbers as integers?\n"]
WARNING:google_genai.types:Warning: there are non-text parts in the response: ['function_call'],returning concatenated text from text parts,check out the non text parts for full response from model.
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2 + 15'}

```

# Output Format Control

In [98]:
from typing import List
from pydantic import BaseModel, Field
from google.genai import types

json_response_config = types.GenerateContentConfig(
  response_mime_type="application/json",
)

class OutputSchema(BaseModel):
    original_query: str = Field(description="The original text from user.")
    corrected_text: str = Field(description="The corrected text.")
    errors: List[str] = Field(description="An array of descriptions of each error.")
    explanations: List[str] = Field(description="An array of explanations for each correction.")

json_schema = OutputSchema.model_json_schema()
json_schema

{'properties': {'original_query': {'description': 'The original text from user.',
   'title': 'Original Query',
   'type': 'string'},
  'corrected_text': {'description': 'The corrected text.',
   'title': 'Corrected Text',
   'type': 'string'},
  'errors': {'description': 'An array of descriptions of each error.',
   'items': {'type': 'string'},
   'title': 'Errors',
   'type': 'array'},
  'explanations': {'description': 'An array of explanations for each correction.',
   'items': {'type': 'string'},
   'title': 'Explanations',
   'type': 'array'}},
 'required': ['original_query', 'corrected_text', 'errors', 'explanations'],
 'title': 'OutputSchema',
 'type': 'object'}

In [99]:
# 1. The {json_schema} in instruction is the key for model to follow the schema.
# 2. The output_schema=OutputSchema provides a validation step after model output.

agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
)

In [100]:
call = await caller_factory(root_agent=agent_grammar, app_name='grammar', user_id='User12345', session_id='333')

In [101]:
session_service.sessions.keys()#['grammar']['User12345']['333']

dict_keys(['hello_search_example'])

In [102]:
pprint_events(call('ho much is three times 5?'))

>>> inside final response...
Final Response (1611.808 ms):
{
  "original_query": "ho much is three times 5?",
  "corrected_text": "How much is three times 5?",
  "errors": [
    "Misspelling"
  ],
  "explanations": [
    "The word 'how' was misspelled. Always remember to double check your spelling to make sure the words are correct!"
  ]
}
-----------------------------
Total elapsed time: 1611.808


In [103]:
pprint_events(call('whut will be 2 plus 3?'))

>>> inside final response...
Final Response (1524.065 ms):
{
  "original_query": "whut will be 2 plus 3?",
  "corrected_text": "What will be 2 plus 3?",
  "errors": [
    "Misspelling",
    "Incorrect word choice"
  ],
  "explanations": [
    "The word 'what' was misspelled. It's important to spell words correctly so others can understand you!",
    "I changed 'be' to 'be' because the sentence is asking a question about what the answer will be."
  ]
}
-----------------------------
Total elapsed time: 1524.065


In [104]:
# session_service.sessions['grammar']['User12345']['333']

# Child Agent Sequential

Concept:
- Flow: sequential

Use case:
- Teaching Assistant

In [105]:
agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
)

simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    # children=[agent_math],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [106]:
from google.adk.agents.sequential_agent import SequentialAgent

agent_teaching_assistant_sequential = SequentialAgent(
    name="agent_teaching_assistant_sequential",
    description="This agent acts as a friendly teaching assistant, checking the grammar of kids' questions, performing math calculations using corrected or original text (if grammatically correct), and providing results or grammar feedback in a friendly tone.",
    sub_agents=[agent_grammar, simple_math_agent],
)

In [107]:
call = await caller_factory(root_agent=agent_teaching_assistant_sequential)

In [108]:
pprint_events(call('what is 1+2?'))

>>> inside final response...
Final Response (2091.507 ms):
{
  "original_query": "what is 1+2?",
  "corrected_text": "What is 1 + 2?",
  "errors": [
    "Missing capitalization at the beginning of the sentence.",
    "Missing spaces around the '+' operator.",
    "Missing question mark at the end of the question."
  ],
  "explanations": [
    "Sentences always start with a capital letter. It's like giving the sentence a name tag!",
    "When you write math problems, it's good to put spaces around the plus sign (+) so it's easier to read. Think of it like giving the numbers some room to breathe!",
    "Questions need a question mark (?) at the end. It's like raising your voice at the end of a question so people know you're asking something!"
  ]
}
-----------------------------


+++ inside the function call...
function, [args]:  add, {'numbers': [1, 2]}
--- inside the function call response...
Function Name: add
Function Results [3]
>>> inside final response...
Final Response (3749.237 ms):
1 + 2 = 3

-----------------------------
Total elapsed time: 3749.237


In [109]:
pprint_events(call('ho much  is three * 2?'))

>>> inside final response...
Final Response (1832.491 ms):
{
  "original_query": "ho much  is three * 2?",
  "corrected_text": "How much is three times two?",
  "errors": [
    "Misspelled 'How'.",
    "Extra space after 'much'.",
    "Used '*' instead of 'times'."
  ],
  "explanations": [
    "'How' needs a 'w' at the beginning. It's like saying, \"Hey, pay attention!\"",
    "Too many spaces make it look messy. Just one space is enough!",
    "The * symbol is okay for computers, but when we write, we usually say 'times'. It's like using our words instead of secret code!"
  ]
}
-----------------------------


+++ inside the function call...
function, [args]:  multiply, {'numbers': [3, 2]}
--- inside the function call response...
Function Name: multiply
Function Results [6]
>>> inside final response...
Final Response (3475.957 ms):
3 * 2 = 6

-----------------------------
Total elapsed time: 3475.957


# Child Agent Loop

Concepts:
- Flow: Loop
- Event.action.escalate

Use case:
- Debate agent?

In [110]:
agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
)

simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    # children=[agent_math],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [113]:
from google.adk.tools.tool_context import ToolContext

def exit_loop(tool_context: ToolContext):
  tool_context.actions.escalate = True

agent_checker = Agent(
    model=MODEL,
    name="agent_checker",
    description="This agent checks if the kid's query is fulfilled. It escalates if both grammar is fixed and the math is calculated",
    instruction="""
    Analyze the chat log between the user (kids) and other agents.

    We are expecting two outcomes from the conversation.
    1. agent_grammar helps to fix the grammar of kid's question.
    2. agent_math_advanced tries to solve the math step by step.

    Decide the action and respond as follows:

    1. If you find user question is incomplete in the chat log, ask a clarification question to the kid. Then call `exit_loop`.

    2. If both agent_grammar and agent_math_advanced have done their task, summarize the answer in a friendly way to the kid. Then call `exit_loop`.

    3. If there is no pending math problem to solve, friendly tell the kid you help fix grammar and solve math, but does not know anything else. Politely ask for math question. Then call exit_loop.

    4. If agent_grammar has fixed the grammar but agent_math_advanced has not generated a final answer with single number, extract corrected_text from most recent JSON response and empahsize that is the math question to solve.

    5. Otherwise, say "Thanks for the question! I'll let grammar and math agent to help you!". Do not call any tools in this case.

    """,
    tools=[exit_loop]
)

agent_extractor = Agent(
    model=MODEL,
    name="agent_extractor",
    description="Extract corrected_text from most recent JSON response and empahsize that is the math question to solve.",
    instruction="""
    Extract corrected_text from most recent JSON response and empahsize that is the math question to solve.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    input_schema=OutputSchema,
)


In [114]:
from google.adk.agents.loop_agent import LoopAgent

agent_teaching_assistant_loop = LoopAgent(
    name="agent_teaching_assistant_loop",
    description="This agent acts as a friendly teaching assistant, checking the grammar of kids' questions, performing math calculations using corrected or original text (if grammatically correct), and providing results or grammar feedback in a friendly tone.",
    sub_agents=[agent_checker, agent_grammar, agent_extractor, agent_math_advanced],
)

In [115]:
call = await caller_factory(root_agent=agent_teaching_assistant_loop)

In [116]:
pprint_events(call('ho much  is three * 2?'))

>>> inside final response...
Final Response (915.446 ms):
Thanks for the question! I'll let grammar and math agent to help you!

-----------------------------
>>> inside final response...
Final Response (2490.818 ms):
{
  "original_query": "ho much  is three * 2?",
  "corrected_text": "How much is three times two?",
  "errors": [
    "Misspelling",
    "Missing word",
    "Incorrect symbol"
  ],
  "explanations": [
    "The word 'how' was misspelled.  It should start with 'h'.",
    "The sentence was missing the word 'times'.",
    "We don't use '*' in a sentence, we say 'times' when we multiply numbers."
  ]
}
-----------------------------
>>> inside final response...
Final Response (3398.582 ms):
The math question to solve is: **How much is three times two?**

-----------------------------
>>> inside final response...
Final Response (4342.898 ms):
<think> The question is three times two, which is 3 * 2 </think>
<call_tool> pass 3 * 2 to simple_math_agent </call_tool>

---------------

+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '3 * 2'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The result is 6.\n"]
>>> inside final response...
Final Response (10565.067 ms):
The result of "3 * 2" is 6

-----------------------------


+++ inside the function call...
function, [args]:  exit_loop, {}
--- inside the function call response...
Function Name: exit_loop
Function Results []
Total elapsed time: 10565.067


In [117]:
pprint_events(call('whut is 1 plus 3 * 5?'))

>>> inside final response...
Final Response (799.575 ms):
Thanks for the question! I'll let grammar and math agent to help you!

-----------------------------
>>> inside final response...
Final Response (2232.261 ms):
{
  "original_query": "whut is 1 plus 3 * 5?",
  "corrected_text": "What is 1 plus 3 times 5?",
  "errors": [
    "Misspelling",
    "Missing word",
    "Incorrect symbol"
  ],
  "explanations": [
    "The word 'what' was misspelled. It should start with 'wh'.",
    "The sentence was missing the word 'times'.",
    "We don't use '*' in a sentence, we say 'times' when we multiply numbers."
  ]
}
-----------------------------
>>> inside final response...
Final Response (3104.98 ms):
The math question to solve is: **What is 1 plus 3 times 5?**

-----------------------------
>>> inside final response...
Final Response (4107.097 ms):
<think> The question is 1 + 3 * 5. I need to calculate 3 * 5 first. </think>
<call_tool> pass 3 * 5 to simple_math_agent </call_tool>

----------

+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '3 * 5'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["15\n"]
>>> inside final response...
Final Response (11075.749 ms):
 The result of 3 * 5 is 15. The question now becomes 1 + 15.
<call_tool> pass 1 + 15 to simple_math_agent </call_tool>

-----------------------------
>>> inside final response...
Final Response (11952.547 ms):
For context:[agent_math_advanced] called tool `simple_math_agent` with parameters: {'request': '1 + 15'}
-----------------------------
>>> inside final response...
Final Response (13405.915 ms):
{
  "original_query": "whut is 1 plus 3 * 5?",
  "corrected_text": "What is 1 plus 3 times 5?",
  "errors": [
    "Misspelling",
    "Missing word",
    "Incorrect symbol"
  ],
  "explanations": [
    "The word 'what' was misspelled. It should start with 'wh'.",
    "The sentence was missing the word 'times'.",
    "We don't use '*' in a sentence, we say 'times' when we multiply numbers."
  ]
}
-----------------------------
>>> ins

+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '1 + 15'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The answer is 16.\n"]
>>> inside final response...
Final Response (17433.403 ms):
The result of "1 + 15" is 16.

-----------------------------


+++ inside the function call...
function, [args]:  exit_loop, {}
--- inside the function call response...
Function Name: exit_loop
Function Results []
Total elapsed time: 17433.403


In [ ]:
#TODO: need more examination of the prompt to make the sequence work.
# call = caller_factory(root_agent=agent_teaching_assistant_loop)

In [ ]:
# pprint_events(call('hello'))

In [ ]:
# pprint_events(call('what is three plus another number?'))

In [ ]:
# pprint_events(call('yup, the other number is 5.'))

# State

Concept:

- session.state -- passing values around.
- session_service.sessions[app_id][user_id][session_is].state
- ToolContext.state
- '{var}' notation in prompt, to access state['var']

Use case:

- Task management

In [ ]:
import random
from google.adk.agents import Agent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.agents.loop_agent import LoopAgent
from google.adk.tools.tool_context import ToolContext
from typing import Tuple

def exit_loop(tool_context: ToolContext):
  tool_context.actions.escalate = True

def roll_dice() -> int:
  """Roll a dice. Return a number between 1 and 6 (inclusive).

  Returns:
      int: A random integer between 1 and 6 (inclusive).
  """
  return random.randint(1, 6)

def update_results(player1: int, player2: int, tool_context: ToolContext) -> str:
  '''Updates the current round of dice rolling results and returns the cumulative results.

  This function takes the results of a single round of dice rolling for two players,
  updates the cumulative scores for each player, and returns the updated cumulative
  scores. It also clears the previous round's results.

  Args:
      player1: The result of player 1's dice roll for the current round.
      player2: The result of player 2's dice roll for the current round.
      tool_context: The tool context object containing the current state.

  Returns:
      A string containing the updated cumulative scores for player 1 and player 2.
  '''
  state = tool_context.state
  if 'play1_total' not in state:
    state['play1_total'] = 0
  state['play1_total'] += player1
  if 'play2_total' not in state:
    state['play2_total'] = 0
  state['play2_total'] += player2
  # Clear the previous round
  state['player_1_result'] = ''
  state['player_2_result'] = ''
  return f"Cumulative points so far. player 1: {state['play1_total']}, player 2: {state['play2_total']}"

player1 = Agent(
    model=MODEL,
    name='player_1',
    description='Play 1',
    instruction='''
    Roll a dice with tool roll_dice. Tell the result in a funny and creative way.
    Do not look at previous rounds result.
    Do not look at at the other player's result.
    Do not respond to the user.
    Only roll your own dice and tell everyone the result.
    ''',
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[roll_dice],
    output_key='player_1_result',
)

player2 = Agent(
    model=MODEL,
    name='player_2',
    description='Play 2',
    instruction='''
    Roll a dice with tool roll_dice. Tell the result in a funny and creative way.
    Do not look at previous rounds result.
    Do not look at at the other player's result.
    Do not respond to the user.
    Only roll your own dice and tell everyone the result.
    ''',
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[roll_dice],
    output_key='player_2_result',
)

judge = Agent(
    model=MODEL,
    name='judge',
    description='Judge',
    instruction='''Analyze both players result and make a short summary of this round.
    Only say the numeric value of player 1 and player 2 result.

    Use the tool `update_results` to add player 1 and player 2 current round results to our scoreboard,
    and get their cumulative results returned by the tool.

    If any player's cumulative result is greater than 15, call `exit_loop` to end the game.
    Otherwise, let the game continue.

    Output in the format:
    - First state both player's current round result
    - Second state both player's cumulative result
    - Third state which player is leading

    Following is your input:

    Play 1 result: {player_1_result}
    Play 2 result: {player_2_result}
    ''',
    tools=[update_results, exit_loop],
)

dice_game = LoopAgent(
    name="dice_game",
    description="A dice game that player 1, player 2 and judge take turns to act.",
    sub_agents=[player1, player2, judge],
)

In [ ]:
call = await caller_factory(root_agent=dice_game, app_name='dice_game', user_id='User12345', session_id='333')

In [ ]:
events = call('start')

In [ ]:
state = session_service.sessions#['dice_game']['User12345']['333'].state
state.keys()

dict_keys(['hello_search_example'])

In [ ]:
start_time = time.time()
for _, event in enumerate(events):
    is_final_response = event.is_final_response()
    function_calls = event.get_function_calls()
    function_responses = event.get_function_responses()
    agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))

    print(f'+++state+++\n{state}\n---state---')

    if is_final_response:
        print('>>> inside final response...')
        final_response = event.content.parts[0].text
        end_time = time.time()
        elapsed_time_ms = round((end_time - start_time) * 1000, 3)
        print(f'Final Response ({elapsed_time_ms} ms):\n{final_response}')
        print("-----------------------------")
    elif function_calls:
        print('+++ inside the function call...')
        for function_call in function_calls:
            print(f"function, [args]:  {function_call.name}, {function_call.args}")
    elif function_responses:
        print('--- inside the function call response...')
        # TODO(Pili): copied from walkthrough codes. Find root cause of 'pending' not found.
        # if not event.actions.pending:
        if True:
            for function_response in function_responses:
                details = function_response.response
                recommended_list = list(details.values())
                print(f"Function Name: {function_response.name}")
                result=json.dumps(recommended_list)
                print(f"Function Results {result}")
        else:
            print(agent_res)
print(f"Total elapsed time: {elapsed_time_ms}")

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

...

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

+++state+++
{'hello_search_example': {'user123456': {'session123456': Session(id='session123456', app_name='hello_search_example', user_id='user123456', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='what is next Google IO date?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-9c4acb2c-33b4-4518-9a79-d8108ff91d5d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='VR3NrINa', timestamp=1748930586.905946), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution

# ADK Web UI



In [118]:
!rm -rf google-adk-101
!git clone https://github.com/hupili/google-adk-101.git

Cloning into 'google-adk-101'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 80 (delta 32), reused 60 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 876.25 KiB | 12.00 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [119]:
!ls google-adk-101/sample_agents/agent_dice

agent.py  __init__.py


In [ ]:
!ls google-adk-101/sample_agents/

In [120]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [121]:
!rm -f localtunnel.log

In [128]:
# Run this command from the shell (lower left corner of CoLab)
# npx localtunnel --port 8000
!npx localtunnel --port 8000 &> localtunnel.log &

In [129]:
!cat localtunnel.log

In [130]:
import time

# wait until the local tunnel is setup

TOTAL_WAIT_SECONDS = 100

for i in range(TOTAL_WAIT_SECONDS):
  output = !cat localtunnel.log
  # print(output)
  if output and output[0].startswith('your url is'):
    print(output[0])
    break
  print(f'waiting local tunnel: {i} / {TOTAL_WAIT_SECONDS} seconds')
  time.sleep(1)

your url is: https://shiny-sloths-behave.loca.lt


In [131]:
# Run this in the CoLab to get the password, which is basically an IP address.
!curl https://loca.lt/mytunnelpassword

34.127.47.201

In [ ]:
# Run this to launch the UI of ADK later
!cd google-adk-101/sample_agents/ && adk web

INFO:     Started server process [21144]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http://localhost:8000.                         |
+-----------------------------------------------------------------------------+

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     182.239.89.235:0 - "GET / HTTP/1.1" 307 Temporary Redirect
INFO:     182.239.89.235:0 - "GET /dev-ui/ HTTP/1.1" 200 OK
INFO:     182.239.89.235:0 - "GET /dev-ui/styles-4VDSPQ37.css HTTP/1.1" 200 OK
INFO:     182.239.89.235:0 - "GET /dev-ui/polyfills-B6TNHZQ6.js HTTP/1.1" 200 OK
INFO:     182.239.89.235:0 - "GET /dev-ui/main-PKDNKWJE.js HTTP/1.1" 200 OK
INFO:     182.239.89.235:0 - "GET /

## Cleanup resources

In [ ]:
!ps aux | grep localtunnel

root        3844  2.5  0.4 1164540 64852 ?       Sl   06:13   0:00 npm exec localtunnel --port 8000
root        3890  3.2  0.4 1164536 65120 ?       Sl   06:13   0:00 npm exec localtunnel --port 8000
root        4007  0.0  0.0   7376  3488 ?        S    06:14   0:00 /bin/bash -c ps aux | grep localtunnel
root        4009  0.0  0.0   6484  2300 ?        S    06:14   0:00 grep localtunnel


In [ ]:
!kill -15 3844 3890

In [ ]:
!ps aux | grep localtunnel

root        4047  0.0  0.0   7376  3500 ?        S    06:14   0:00 /bin/bash -c ps aux | grep localtunnel
root        4049  0.0  0.0   6484  2340 ?        S    06:14   0:00 grep localtunnel
